In [1]:
import os
from glob import glob
import numpy as np
import sys
import pydicom
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# from FindPoints.detect_points import detect_spheres_from_dicom
from detect_balls import detect_spheres_from_dicom
from cal_math import rigidTransform3D, transform3d, match_balls_by_geometry, Q2M
from plan_rom_data import plan_ball, plan_robot, plan_body, rom_body, rom_robot

[Taichi] version 1.7.3, llvm 15.0.1, commit 5ec301be, win, python 3.8.10
[Taichi] Starting on arch=x64


In [2]:
# 针尖法向量计算
rom_plane_robot_mat, _=rigidTransform3D(plan_robot[:4].T, rom_robot.T)  # 模型到rom的变换矩阵
# print(transform3d(plan_robot[:4], rom_plane_robot_mat))
# print(np.linalg.norm(transform3d(plan_robot[:4],rom_plane_robot_mat)-rom_robot,axis=1))  # 光球模型在rom下的位置和误差
r_pos = transform3d(plan_robot[4:],rom_plane_robot_mat)
r_plan_dis = r_pos[0] - r_pos[1]
tail_x = r_plan_dis / np.linalg.norm(r_plan_dis)
tail_p = r_pos[0]
tail_x, tail_p

(array([-0.7186764 , -0.41793381,  0.55572976]),
 array([43.38508775, -4.95764079,  4.6531603 ]))

In [4]:
def find_ct_ball(dicom_folder):
    dicom_paths = glob(os.path.join(dicom_folder, "*.dcm"))
    dicom_files = [pydicom.dcmread(path) for path in dicom_paths] 
    def get_z_position(dcm):
        if hasattr(dcm, 'SliceLocation'):
            # print("paixun 1")
            return float(dcm.SliceLocation)  # 优先使用 SliceLocation
        elif hasattr(dcm, 'ImagePositionPatient'):
            # print("paixun 2")
            return float(dcm.ImagePositionPatient[2])  # 否则使用 ImagePositionPatient 的 z 分量
        else:
            # print("paixun 3")
            return 0  # 如果没有 z 信息，默认 0
    dicom_files.sort(key=get_z_position)
    print(type(dicom_files))
    slices=dicom_files
    data = np.stack([s.pixel_array for s in slices])
    pixel_spacing = slices[0].PixelSpacing
    if hasattr(slices[0], 'SpacingBetweenSlices'):
        thickness = slices[0].SpacingBetweenSlices
    else:
        thickness = abs(slices[1].ImagePositionPatient[2] - slices[0].ImagePositionPatient[2])
    # print("体厚度=", thickness, pixel_spacing)
    #数据处理
    pos_ori = slices[-1].ImagePositionPatient
    data = np.flip(data, axis=0)
    axis = slices[0].ImageOrientationPatient
    print("PatientID:",slices[0].PatientID)
    #find points
    # print(f"坐标系={axis}")
    row_cosines = np.array(axis[0:3])
    col_cosines = np.array(axis[3:6])

    spheres = detect_spheres_from_dicom(data=data, pixel_spacing=pixel_spacing, thickness=thickness, pos_ori=pos_ori,
                                        r_range=0.75, rmax=5, rlen=10, output_points=15)
    spheres[:,0]=-pos_ori[0]+spheres[:,0] * pixel_spacing[0]*row_cosines[0]*-1
    spheres[:,1]=-pos_ori[1]+spheres[:,1] * pixel_spacing[1]*col_cosines[1]*-1
    spheres[:,2]=pos_ori[2]-spheres[:,2] * thickness
    return spheres


In [5]:
#规划
dicom_folder = "C:/Users/YangLiangZhu/Desktop/泰州精度验证/Tai0605/8/CT-MP25075-250605-6gui/S8010"
ct_all_ball = find_ct_ball(dicom_folder)
matched_points, _, _ = match_balls_by_geometry(plan_ball, ct_all_ball[:, :3], verbose=True)
ct_ball = matched_points[:, :3]
ct_plan_ball_mat, _ = rigidTransform3D(plan_ball.T, ct_ball.T)
ct_plan_ball_body = transform3d(plan_body, ct_plan_ball_mat)
T0, _ = rigidTransform3D(rom_body.T, ct_plan_ball_body.T)


<class 'list'>
PatientID: CT-MP25075-250605-6
最佳匹配索引组合： [1, 0, 2, 3]
对应的检测点（已按实际顺序匹配）：
实际球 1 ⟶ 检测点 [  22.398438   20.848688 -398.176   ]
实际球 2 ⟶ 检测点 [  23.5        14.973688 -372.176   ]
实际球 3 ⟶ 检测点 [  16.523438     7.9971256 -335.176    ]
实际球 4 ⟶ 检测点 [  -6.2421875   11.669001  -339.176    ]
总距离误差： 0.9200575094580724


In [7]:

# 验证
ver_dicom_folder = "C:/Users/YangLiangZhu/Desktop/泰州精度验证/Tai0605/8/CT-MP25075-250605-yan/S10010"

# print(time.time() - ts)
ver_ct_all_ball=find_ct_ball(ver_dicom_folder)

print(ver_ct_all_ball[:5, :3])
ver_matched_points, _, _ = match_balls_by_geometry(plan_ball, ver_ct_all_ball[:, :3],                                                               verbose=True)
ver_ct_ball = ver_matched_points[:, :3]
ver_ct_plan_ball_mat, _ = rigidTransform3D(plan_ball.T, ver_ct_ball.T)
# print("模型到CT坐标的刚性变换矩阵", ct_plan_ball_mat)

ver_ct_plan_ball_body = transform3d(plan_body, ver_ct_plan_ball_mat)
print("ct下光球位置", ver_ct_plan_ball_body)

ver_T0, _ = rigidTransform3D(rom_body.T, ver_ct_plan_ball_body.T)
rom_body, ver_T0

<class 'list'>
PatientID: CT-MP25075-250605-8
[[  15.7890625    7.9971256 -335.176    ]
 [  22.03125     21.215876  -398.176    ]
 [  23.5         14.973688  -372.176    ]
 [  -6.2421875   12.036188  -339.176    ]
 [ -87.02344     80.33306   -112.176    ]]
最佳匹配索引组合： [1, 2, 0, 3]
对应的检测点（已按实际顺序匹配）：
实际球 1 ⟶ 检测点 [  22.03125    21.215876 -398.176   ]
实际球 2 ⟶ 检测点 [  23.5        14.973688 -372.176   ]
实际球 3 ⟶ 检测点 [  15.7890625    7.9971256 -335.176    ]
实际球 4 ⟶ 检测点 [  -6.2421875   12.036188  -339.176    ]
总距离误差： 0.5367003472622744
ct下光球位置 [[  20.78681717   22.71823742 -420.53881737]
 [  20.67703209    3.08791127 -386.02956872]
 [  23.18611756    9.26190849 -362.11647088]
 [   2.8532238    -3.54858085 -340.17129684]]


(array([[  0.  ,   0.  ,   0.  ],
        [-10.76,  38.02,   0.  ],
        [  0.  ,  60.42,   0.  ],
        [ -8.63,  83.9 , -20.23]]),
 array([[ 9.14434841e-02,  3.68028371e-02,  9.95129962e-01,
          2.06742824e+01],
        [ 9.69704628e-01, -2.30607955e-01, -8.05785635e-02,
          2.26852498e+01],
        [ 2.26519365e-01,  9.72350514e-01, -5.67754763e-02,
         -4.20727726e+02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]))

In [10]:
# CT里手点针尖的位置
ct_find_needle_tips = {1: np.array([-11.568, 81.232, -228.932]), 
                       2: np.array([17.889, 84.551, -226.954]),
                       3: np.array([-18.696, 81.497, -256.153]),
                       4: np.array([14.037, 84.261, -278.169]),
                       5: np.array([20.644, 85.943, -222.288]),
                       6: np.array([-10.115, 86.031, -204.159]),
                       7: np.array([18.342, 92.726, -200.170]),
                       8: np.array([-16.345, 81.005, -230.308])}

ct_find_needle_tails = {1: np.array([-46.617, 9.498, -227.164]),
                        2: np.array([38.553, 0.705, -232.203]),
                        3: np.array([-50.404, 15.009, -254.229]),
                        4: np.array([41.095, -6.627, -260.023]),
                        5: np.array([30.264, -17.531, -195.207]),
                        6: np.array([-60.872, -29.157, -180.259]),
                        7: np.array([35.987, 1.184, -197.973]),
                        8: np.array([-56.234, -28.724, -215.179])}

In [11]:
# REDIS数据
pose_to_ct = {1: np.array([[-11.478202161858752,
                            91.7897037540662,
                            -231.09500122070304],
                           [-52.37707819658013,
                            -8.884448298738455,
                            -225.70174284249748]]),
              2: np.array([[16.17977513461503,
                            92.49141776485129,
                            -232.6759948730469],
                           [39.999999638353835,
                            -5.486105215110314,
                            -232.67599487304688]]),
              3: np.array([
                [
                    -14.382022341880003,
                    92.04197958604414,
                    -259.57487126941436],
                [
                    -57.977527565703895,
                    -9.531048824374778,
                    -252.3838600984744
                  ]]),
              4: np.array([
                  [
                    15.280898738247528,
                    92.04197958604414,
                    -288.6759948730469
                ],
                  [
                      45.8426962147426,
                      -25.710823261432665,
                      -254.51869181108177
                  ]]),
              5: np.array([
                  [
                      17.078651530982537,
                      87.54759779797251,
                      -231.32768027849565
                  ],
                  [
                      34.60674126014882,
                      -32.452395943540125,
                      -190.42880424377427
                  ]
              ]),
              6: np.array([
                  [
                      -13.033707747328764,
                      97.43523773173011,
                      -201.67599487304688
                  ],
                  [
                      -57.977527565703895,
                      -23.463632367396855,
                      -181.90071415296183
                  ]]
              ),
              7: np.array([[
                  17.078651530982537,
                  95.18804683769429,
                  -201.67599487304688
              ],
                  [
                      40.89887603472134,
                      -31.553519585925798,
                      -196.28273649484186
                  ]
              ]),
              8: np.array([[
                  -13.483145945512536,
                  94.28917048007995,
                  -225.6759948730469
              ],
                  [
                      -46.7415726111101,
                      -9.980487003181949,
                      -217.13666910755563
                  ]])
              }

robot_after = {1: np.array([-163.148193359375,
                            212.99407958984375,
                            -1651.0927734375,
                            0.8833289742469788,
                            -0.3302684724330902,
                            -0.32147645950317383,
                            -0.08547268807888031]),
               2: np.array([-130.24606323242188,
                            218.477783203125,
                            -1527.9007568359375,
                            0.8530240058898926,
                            -0.5010212063789368,
                            -0.055626656860113144,
                            0.13503168523311615]),
               3: np.array([-161.05067443847656,
                            187.23045349121094,
                            -1652.1734619140625,
                            0.871877908706665,
                            -0.34895244240760803,
                            -0.333023339509964,
                            -0.08459670841693878]),
               4: np.array([-208.33087158203125,
                            77.00535583496094,
                            -1611.321044921875,
                            0.9560052752494812,
                            -0.27341169118881226,
                            -0.10555561631917953,
                            0.012565813958644867]),
               5: np.array([
                   -140.6206817626953,
                   255.92921447753906,
                   -1559.7490234375,
                   0.820655107498169,
                   -0.5669674277305603,
                   -0.05909746140241623,
                   -0.039758335798978806
               ],
               )
               }

spine_after = {1: np.array([-33.34297561645508,
                            43.048763275146484,
                            -1528.234619140625,
                            0.9649072885513306,
                            -0.06590159237384796,
                            0.22094786167144775,
                            0.12567009031772614]),
               2: np.array([-31.724733352661133,
                            42.38328170776367,
                            -1529.2227783203125,
                            0.9648662805557251,
                            -0.06486830860376358,
                            0.22007229924201965,
                            0.12803645431995392]),
               3: np.array([-31.7352237701416,
                            45.380958557128906,
                            -1530.553955078125,
                            0.9648716449737549,
                            -0.06484962999820709,
                            0.2196609377861023,
                            0.1287100613117218]),
               4: np.array([-32.76726531982422,
                            42.52442169189453,
                            -1529.698974609375,
                            0.965252161026001,
                            -0.06576952338218689,
                            0.21870079636573792,
                            0.12701416015625]),
               5: np.array([
                   -34.308326721191406,
                   29.772933959960938,
                   -1530.30810546875,
                   0.9653386473655701,
                   -0.07120106369256973,
                   0.21724843978881836,
                   0.12591595947742462
               ]
               )
               }

robot_before = {1: np.array([-163.148193359375,
                             212.99407958984375,
                             -1651.0927734375,
                             0.8833289742469788,
                             -0.3302684724330902,
                             -0.32147645950317383,
                             -0.08547268807888031]),
                2: np.array([-130.1228485107422,
                             218.465576171875,
                             -1527.8455810546875,
                             0.8528672456741333,
                             -0.5013030171394348,
                             -0.056207798421382904,
                             0.13473451137542725]),
                3: np.array([
                    -160.98760986328125,
                    188.01165771484375,
                    -1653.105224609375,
                    0.8726862072944641,
                    -0.3505486249923706,
                    -0.33052995800971985,
                    -0.07927476614713669
                ]),
                4: np.array([-125.41316223144531,
                             196.4991455078125,
                             -1532.4453125,
                             0.8343876004219055,
                             -0.5509225726127625,
                             -0.013552417047321796,
                             -0.009897115640342236]),
                5: np.array([-140.59359741210938,
                             256.26007080078125,
                             -1560.08203125,
                             0.820141077041626,
                             -0.5676160454750061,
                             -0.059834908694028854,
                             -0.040003687143325806]),
                6: np.array([-172.16827392578125,
                             251.9439697265625,
                             -1658.2611083984375,
                             0.8537662625312805,
                             -0.41335898637771606,
                             -0.3004518151283264,
                             -0.09973087906837463]),
                7: np.array([-142.87127685546875,
                             263.3289794921875,
                             -1541.806640625,
                             0.8945340514183044,
                             -0.4429192841053009,
                             -0.05515580251812935,
                             0.024272993206977844]),
                8: np.array([-175.69700622558594,
                             215.04905700683594,
                             -1642.80615234375,
                             0.8401060104370117,
                             -0.4507373571395874,
                             -0.30065783858299255,
                             -0.025740014389157295])
                }

spine_before = {1: np.array([-33.34297561645508,
                             43.048763275146484,
                             -1528.234619140625,
                             0.9649072885513306,
                             -0.06590159237384796,
                             0.22094786167144775,
                             0.12567009031772614]),
                2: np.array([-33.437232971191406,
                             42.31951904296875,
                             -1528.57470703125,
                             0.9649953842163086,
                             -0.06566346436738968,
                             0.2206740379333496,
                             0.12559938430786133]),
                3: np.array([
                    -33.1001091003418,
                    45.30834197998047,
                    -1530.1527099609375,
                    0.9653571248054504,
                    -0.06510182470083237,
                    0.2181934416294098,
                    0.12743228673934937]),
                4: np.array([-32.68147277832031,
                             28.49115753173828,
                             -1530.2279052734375,
                             0.9647408127784729,
                             -0.07046534866094589,
                             0.21886256337165833,
                             0.1280975043773651]),
                5: np.array([-33.02286148071289,
                             29.723434448242188,
                             -1530.746337890625,
                             0.9649015665054321,
                             -0.07060898095369339,
                             0.21842744946479797,
                             0.1275491565465927]),
                6: np.array([-35.30064392089844,
                             39.158348083496094,
                             -1530.2076416015625,
                             0.9657207131385803,
                             -0.0676775872707367,
                             0.21681463718414307,
                             0.12567667663097382]),
                7: np.array([-36.772430419921875,
                             42.181114196777344,
                             -1530.517578125,
                             0.9660152196884155,
                             -0.06780268251895905,
                             0.21599169075489044,
                             0.1247594952583313]),
                8: np.array([-35.125118255615234,
                             42.87014389038086,
                             -1531.5341796875,
                             0.9656884670257568,
                             -0.06743000447750092,
                             0.2164064347743988,
                             0.1267566978931427])
                }



In [12]:
# change1
spine = Q2M(spine_before[8][3:], spine_before[8][:3])
robot = Q2M(robot_before[8][3:], robot_before[8][:3])


In [13]:
# change2
ver_ct_ball_mat, _ = rigidTransform3D(ct_ball.T, ver_ct_ball.T)
ver_ct_ball_hope_needle = transform3d(pose_to_ct[8], ver_ct_ball_mat)
ver_t0_needle=transform3d(pose_to_ct[8], np.linalg.inv(T0)@ver_T0)
ver_ct_ball_hope_needle[0], ver_t0_needle[0]

(array([ -11.99578022,   93.94047452, -225.01609048]),
 array([  -9.45459548,   93.58353455, -226.35236666]))

In [14]:
# change3  验证的和ct的
ver_ct_ball_hope_needle[0]- ct_find_needle_tips[8] ,np.linalg.norm(ver_ct_ball_hope_needle[0] - ct_find_needle_tips[8])

(array([ 4.34921978, 12.93547452,  5.29190952]), 14.637162295552507)

In [15]:
# change4 希望扎针
hope_X_dis = pose_to_ct[8][1] - pose_to_ct[8][0]
hope_X_dis, hope_X_dis/np.linalg.norm(hope_X_dis)

(array([ -33.25842667, -104.26965748,    8.53932577]),
 array([-0.30296082, -0.94982308,  0.07778724]))

In [16]:
#规划robot扎针
ca_robot_x = robot[:3,:3]@np.array(tail_x)
ct_robot_x = (T0@np.linalg.inv(spine)@robot)[:3,:3]@np.array(tail_x)
ct_robot_p = transform3d(np.array(tail_p).reshape(1,3),T0@np.linalg.inv(spine)@robot)
ct_robot_x, ct_robot_p

(array([-0.30384035, -0.94944145,  0.07900612]),
 array([[ -55.92047934,  -38.88044613, -214.70807333]]))

In [13]:
from cal_math import calculate_angle
calculate_angle(hope_X_dis, ct_robot_x)  # 规划和验证的

11.421288136967506

In [14]:
# change5 CT 扎针 
hand_ct_X_dis = ct_find_needle_tails[8] - ct_find_needle_tips[8]
hand_ct_X_dis, hand_ct_X_dis / np.linalg.norm(hand_ct_X_dis)

(array([ -39.889, -109.729,   15.129]),
 array([-0.33881617, -0.93203539,  0.12850535]))

In [ ]:
calculate_angle(hand_ct_X_dis, ct_robot_x)  # 验证的和规划的

NameError: name 'calculate_angle' is not defined